<a href="https://colab.research.google.com/github/RohanCoderiiitb/FFT-Processor/blob/main/FFT_Processor_Evaluation_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparission between FFT results for data in FP4 and FP8 format

We are comparing results obtained by using FP4(E2M1), FP4(E1M2), FP8 with those of FP32.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def quantize_fp_custom(x, exp_bits, mant_bits, bias):
    """
    Hardware-accurate quantization for custom FP formats.
    Supports saturation and flushing underflows to zero.
    """
    if x == 0: return 0.0
    sign = np.sign(x)
    val = abs(x)

    exp = np.floor(np.log2(val))

    max_exp = (2**exp_bits - 1) - bias
    if exp > max_exp:
        max_val = (2**max_exp) * (1 + (2**mant_bits - 1) / 2**mant_bits)
        return sign * max_val

    min_exp = 1 - bias
    if exp < min_exp:
        return 0.0

    mant = val / (2**exp) - 1.0
    mant_q = np.round(mant * (2**mant_bits)) / (2**mant_bits)

    return sign * (1 + mant_q) * (2**exp)

def quant_e2m1(x): return quantize_fp_custom(x, exp_bits=2, mant_bits=1, bias=1)
def quant_e1m2(x): return quantize_fp_custom(x, exp_bits=1, mant_bits=2, bias=0)
def quant_fp8(x):  return quantize_fp_custom(x, exp_bits=4, mant_bits=3, bias=7)

In [ ]:
def fft_radix2_hw_accurate(x, quant_func):
    x = x.astype(np.complex64).copy()
    N = len(x)
    stages = int(np.log2(N))

    for s in range(stages):
        m = 2**(s+1)
        for k in range(0, N, m):
            for j in range(m//2):
                w_ideal = np.exp(-2j * np.pi * j / m)
                wr_q = quant_func(w_ideal.real)
                wi_q = quant_func(w_ideal.imag)
                W_q = complex(wr_q, wi_q)

                item = x[k+j+m//2]
                real_part = quant_func(item.real * W_q.real) - quant_func(item.imag * W_q.imag)
                imag_part = quant_func(item.real * W_q.imag) + quant_func(item.imag * W_q.real)
                t = complex(quant_func(real_part), quant_func(imag_part))

                a = x[k+j]

                x[k+j] = complex(quant_func(a.real + t.real), quant_func(a.imag + t.imag))
                x[k+j+m//2] = complex(quant_func(a.real - t.real), quant_func(a.imag - t.imag))

    return x

## Testing using the following signals:
1. Single tone sinusoid
2. Dual tone sinusoid

Signal to Quantization Noise Ratio(SQNR) is the metric used to analyse the performance of different floating point architectures

### Single Tone Sinusoid

In [ ]:
N = 32
k = 3
n = np.arange(N)
x = np.sin(2*np.pi*k*n/N)

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x,quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x,quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x,quant_fp8)

def compute_sqnr(ref, quant):
  signal_power = np.sum(np.abs(ref)**2)
  noise_power = np.sum(np.abs(ref - quant)**2)
  return 10*np.log10(signal_power/noise_power)

print("SQNR Analysis:")
print(f"FP4(E2M1): {compute_sqnr(X_fp4_e2m1, X_fft_ref)}")
print(f"FP4(E1M2): {compute_sqnr(X_fp4_e1m2, X_fft_ref)}")
print(f"FP8(E4M3): {compute_sqnr(X_fp8_e4m3, X_fft_ref)}")

SQNR Analysis:
FP4(E2M1): -16.40150040936102
FP4(E1M2): -inf
FP8(E4M3): -3.049955517874458


/tmp/ipython-input-479705848.py:15: RuntimeWarning: divide by zero encountered in log10
  return 10*np.log10(signal_power/noise_power)


### Dual Tone Sinusoid

In [ ]:
x = (np.sin(2*np.pi*3*n/N) + 0.5*np.sin(2*np.pi*5*n/N))
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

print("SQNR Analysis:")
print(f"FP4(E2M1): {compute_sqnr(X_fft_ref, X_fp4_e2m1)}")
print(f"FP4(E1M2): {compute_sqnr(X_fft_ref, X_fp4_e1m2)}")
print(f"FP8(E4M3): {compute_sqnr(X_fft_ref, X_fp8_e4m3)}")

SQNR Analysis:
FP4(E2M1): -1.1195972128452403
FP4(E1M2): 0.0
FP8(E4M3): -2.4179822686366377


### Strong + Weak Signal

In [ ]:
x = (1.0 * np.sin(2*np.pi*3*n/N) + 0.1 * np.sin(2*np.pi*12*n/N))
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

print("SQNR Analysis:")
print(f"FP4(E2M1): {compute_sqnr(X_fft_ref, X_fp4_e2m1)}")
print(f"FP4(E1M2): {compute_sqnr(X_fft_ref, X_fp4_e1m2)}")
print(f"FP8(E4M3): {compute_sqnr(X_fft_ref, X_fp8_e4m3)}")

SQNR Analysis:
FP4(E2M1): -0.8586042779764633
FP4(E1M2): 0.0
FP8(E4M3): -2.394800224659955


### White Gaussian Noise

In [ ]:
np.random.seed(42)
x = np.random.normal(0, 0.5, N)
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

print("SQNR Analysis:")
print(f"FP4(E2M1): {compute_sqnr(X_fft_ref, X_fp4_e2m1)}")
print(f"FP4(E1M2): {compute_sqnr(X_fft_ref, X_fp4_e1m2)}")
print(f"FP8(E4M3): {compute_sqnr(X_fft_ref, X_fp8_e4m3)}")

SQNR Analysis:
FP4(E2M1): -0.8879950694438441
FP4(E1M2): 0.0
FP8(E4M3): -2.1760971139527663


### Impulse Signal

In [ ]:
x = np.zeros(N)
x[0] = 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

print("SQNR Analysis:")
print(f"FP4(E2M1): {compute_sqnr(X_fft_ref, X_fp4_e2m1)}")
print(f"FP4(E1M2): {compute_sqnr(X_fft_ref, X_fp4_e1m2)}")
print(f"FP8(E4M3): {compute_sqnr(X_fft_ref, X_fp8_e4m3)}")

SQNR Analysis:
FP4(E2M1): inf
FP4(E1M2): inf
FP8(E4M3): inf


/tmp/ipython-input-479705848.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  return 10*np.log10(signal_power/noise_power)
